In [1]:
import polars as pl

DATA_FILE = "2024_fb_posts_president_scored_anon.csv"

# Load and deduplicate columns
df = pl.read_csv(DATA_FILE)
df = df.select([pl.col(c) for i, c in enumerate(df.columns) if c not in df.columns[:i]])

# Determine column types
numeric_cols = [col for col in df.columns if df[col].dtype in (pl.Float64, pl.Int64)]
non_numeric_cols = [col for col in df.columns if df[col].dtype == pl.Utf8]

# === Summary Printer ===
def summarize(df_part: pl.DataFrame, label: str):
    print(f"\n{'=' * 60}\n{label}\n{'=' * 60}")

    if df_part.is_empty():
        print("No data available.\n")
        return

    if numeric_cols:
        print("\n📊 Numeric Summary:")
        numeric_data = df_part.select([pl.col(col).cast(pl.Float64) for col in numeric_cols])
        print(numeric_data.describe())

    print("\n📝 Categorical Summary:")
    for col in non_numeric_cols:
        unique_vals = df_part.select(pl.col(col).n_unique()).item()
        vc = df_part.select(pl.col(col).value_counts()).unnest(col)
        if vc.height > 0:
            most_common = vc[0, col]
            most_count = vc[0, "count"]
            print(f"- {col}:")
            print(f"    Unique values : {unique_vals}")
            print(f"    Most frequent : '{most_common}' ({most_count} times)'")
    print()

# === Overall Summary ===
summarize(df, "🌐 Overall Facebook Posts Summary")

# === Grouped by Facebook_Id (first 5)
if "Facebook_Id" in df.columns:
    unique_ids = df.select("Facebook_Id").unique().head(5)["Facebook_Id"]
    for fb_id in unique_ids:
        group_df = df.filter(pl.col("Facebook_Id") == fb_id)
        summarize(group_df, f"📁 Grouped by Facebook_Id = {fb_id}")

# === Grouped by Facebook_Id + post_id (top 10 combos)
if all(col in df.columns for col in ["Facebook_Id", "post_id"]):
    top_10 = (
        df.group_by(["Facebook_Id", "post_id"])
        .agg(pl.len().alias("count"))
        .sort("count", descending=True)
        .head(10)
    )

    for row in top_10.iter_rows():
        fb_id, post_id, _ = row
        group_df = df.filter((pl.col("Facebook_Id") == fb_id) & (pl.col("post_id") == post_id))
        summarize(group_df, f"🔗 Grouped by Facebook_Id = {fb_id}, post_id = {post_id}")


🌐 Overall Facebook Posts Summary

📊 Numeric Summary:
shape: (9, 41)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆ Likes     ┆ Comments  ┆ Shares    ┆ … ┆ incivilit ┆ scam_illu ┆ freefair_ ┆ fraud_il │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ y_illumin ┆ minating  ┆ illuminat ┆ luminati │
│ str       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ ating     ┆ ---       ┆ ing       ┆ ng       │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ f64       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ f64       ┆           ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 19009.0   ┆ 19009.0   ┆ 19009.0   ┆ … ┆ 19009.0   ┆ 18060.0   ┆ 19009.0   ┆ 19009.0  │
│ null_coun ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ … ┆ 0.0       ┆ 949.0     ┆ 0.0       ┆ 0.0      │
│ t         ┆         